## Objetivos formativos

- Resolver un problema de clasificación de Spam
- Proponer diferentes métodos de combinación de árboles: bootstrap y random forests


# Ensembles de árboles en un dataset de detección de spam

Si bien en anteriores prácticas hemos insistido en emplear entre otros el dataset de digits, es bien conocido que para este tipo de datasets que representan imágenes, no es lo habitual emplear árboles de decisión.

Árboles de decisión y todos los ensembles propuestos funcionan en la práctica muy bien en datos tabulares como puede ser el problema de reconocimiento de spam en mails.

Se propone pues emplear el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.


## Un primer clasificador con un único árbol de decisión

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


## Grid Search
G = {"max_depth": [2,4,6,8,10], "min_samples_split": [2,3,4]}

GS = GridSearchCV(DecisionTreeClassifier(random_state=23), G, scoring='accuracy', refit=True, cv=5)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

(4601, 57)
Precisión: 93.9% con {'max_depth': 10, 'min_samples_split': 4}


## Bagging

Bagging es una técnica genérica de muestreo con remplanzamiento del conjunto de muestras de entrenamiento. Se puede emplear junto con cualquier clasificador para obtener un ensemble. Los parámetros más importantes son por un lado el estimador que se quiere emplear, en nuestro caso un DecisionTree, y el número de estimadores que se quiere combinar en el ensemble.

In [2]:
from sklearn.ensemble import BaggingClassifier

clf= BaggingClassifier(estimator=DecisionTreeClassifier(random_state=23,max_depth=10,min_samples_split=4),n_estimators=100)

acc = clf.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%}')




Precisión: 94.9%


**Ejercicio:** Encuentra los mejores parámetros mediante grid search. En concreto, n_estimators y max_depth de los árboles de decisión empleados. Se debería poder llegar alrededor de un **94.5% de acierto**.  

In [7]:
### SOLUCION
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

DT = DecisionTreeClassifier(random_state=23)
bagging_clf = BaggingClassifier(estimator=DT, random_state=42)

G = {
    'estimator__max_depth': [18, 20],  # del DT
    'estimator__min_samples_split': [2, 3], # del DT
    'n_estimators': [50], # del Bagging
    #'max_samples': [0.8, 1.0],  # del Bagging
    #'max_features': [0.8, 1.0], # del Bagging
}

# Configurar GridSearchCV
grid_search = GridSearchCV(bagging_clf, G, scoring='accuracy', cv=5,  refit=True, verbose=10)

# Ajustar el GridSearchCV
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y la precisión en el conjunto de prueba
print(f"Mejores parámetros: {grid_search.best_params_}")
best_clf = grid_search.best_estimator_

# Evaluar en el conjunto de prueba
y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión en el conjunto de prueba: {accuracy:.1%}')


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50
[CV 1/5; 1/4] END estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50;, score=0.938 total time=   1.0s
[CV 2/5; 1/4] START estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50
[CV 2/5; 1/4] END estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50;, score=0.931 total time=   1.1s
[CV 3/5; 1/4] START estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50
[CV 3/5; 1/4] END estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50;, score=0.947 total time=   1.0s
[CV 4/5; 1/4] START estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50
[CV 4/5; 1/4] END estimator__max_depth=18, estimator__min_samples_split=2, n_estimators=50;, score=0.933 total time=   1.0s
[CV 5/5; 1/4] START estimator__max_depth=18, estimator__min_samples_spli

## RandomForest

RandomForest es otra técnica de ensemble, en este caso no genérica sino particular para árboles de decisión. Cada árbol se construye empleando una selección aleatoria de las características en cada uno de los nodos. El parámetro más importante es "n_estimators", el número de árboles que se combinan. El parámetro binario "bootstrap" controla también si se realiza bootstrap de las muestras (Bagging) o no.



In [10]:
from sklearn.ensemble import RandomForestClassifier

clf= RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_split=4, bootstrap=False, random_state=23)

acc = clf.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%}')



Precisión: 94.1%


**Ejercicio:** Realiza una búsqueda de parámetros mediande grid search. En concreto para los parámetros: n_estimators, max_depth y bootstrap. Se debería poder llegar alrededor de un **95% de acierto**.  

In [15]:
# Solucion
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define el modelo base
RF = RandomForestClassifier(random_state=23)

# Define el espacio de búsqueda de hiperparámetros
G = {
    'n_estimators': [10, 50, 100, 200],  # Número de árboles
    'max_depth': [5, 10, 20, None],  # Profundidad máxima del árbol
    #'min_samples_split': [2, 4, 8],  # Muestras mínimas para dividir un nodo
    'max_features': ['sqrt', 'log2', 0.5],  # Máximo número de características consideradas
    'bootstrap': [True, False],  # Si se emplea muestreo con reemplazo
    #'criterion': ['gini', 'entropy'],  # Criterio para medir la calidad de una división
}

# Configurar GridSearchCV
grid_search = GridSearchCV(RF, G, scoring='accuracy', cv=5, refit=True, verbose=10)

# Ajustar el GridSearchCV
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y la precisión en el conjunto de prueba
print(f"Mejores parámetros: {grid_search.best_params_}")
best_rf_clf = grid_search.best_estimator_

# Evaluar en el conjunto de prueba
y_pred = best_rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión en el conjunto de prueba: {accuracy:.1%}')


Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5; 1/96] START bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10
[CV 1/5; 1/96] END bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10;, score=0.913 total time=   0.0s
[CV 2/5; 1/96] START bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10
[CV 2/5; 1/96] END bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10;, score=0.912 total time=   0.0s
[CV 3/5; 1/96] START bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10
[CV 3/5; 1/96] END bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10;, score=0.931 total time=   0.0s
[CV 4/5; 1/96] START bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10
[CV 4/5; 1/96] END bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10;, score=0.925 total time=   0.0s
[CV 5/5; 1/96] START bootstrap=True, max_depth=5, max_features=sqrt, n_estimators=10
[CV 5/5; 1/96] END bootstrap=True, max_depth=5, m

In [ ]:
# Solucion
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define el modelo base
RF = RandomForestClassifier(random_state=23)

# Define el espacio de búsqueda de hiperparámetros
G = {
    'n_estimators': [200,300,400,500,750,1000],  # Número de árboles
    'max_depth': [None],  # Profundidad máxima del árbol
    #'min_samples_split': [2, 4, 8],  # Muestras mínimas para dividir un nodo
    'max_features': ['log2'],  # Máximo número de características consideradas
    'bootstrap': [True],  # Si se emplea muestreo con reemplazo
    #'criterion': ['gini', 'entropy'],  # Criterio para medir la calidad de una división
}

# Configurar GridSearchCV
grid_search = GridSearchCV(RF, G, scoring='accuracy', cv=5, refit=True, verbose=10)

# Ajustar el GridSearchCV
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y la precisión en el conjunto de prueba
print(f"Mejores parámetros: {grid_search.best_params_}")
best_rf_clf = grid_search.best_estimator_

# Evaluar en el conjunto de prueba
y_pred = best_rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión en el conjunto de prueba: {accuracy:.1%}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START bootstrap=True, max_depth=None, max_features=log2, n_estimators=200
[CV 1/5; 1/4] END bootstrap=True, max_depth=None, max_features=log2, n_estimators=200;, score=0.951 total time=   0.4s
[CV 2/5; 1/4] START bootstrap=True, max_depth=None, max_features=log2, n_estimators=200
[CV 2/5; 1/4] END bootstrap=True, max_depth=None, max_features=log2, n_estimators=200;, score=0.954 total time=   0.4s
[CV 3/5; 1/4] START bootstrap=True, max_depth=None, max_features=log2, n_estimators=200
[CV 3/5; 1/4] END bootstrap=True, max_depth=None, max_features=log2, n_estimators=200;, score=0.962 total time=   0.4s
[CV 4/5; 1/4] START bootstrap=True, max_depth=None, max_features=log2, n_estimators=200
[CV 4/5; 1/4] END bootstrap=True, max_depth=None, max_features=log2, n_estimators=200;, score=0.946 total time=   0.4s
[CV 5/5; 1/4] START bootstrap=True, max_depth=None, max_features=log2, n_estimators=200
[CV 5/5; 1/4] END bootst